# Office Automation Project

## Introduction
This is a collection of scripts in Visual Basic and Power Shell to automate some daily office tasks and save workhours, every minute counts! 

### Prerequisites
- PowerShell
- Visual Basic Scripts

### Retrieving Computer Info
The PowerShell script below, <i>Serial_Number.ps1,</i> collects the current date, computer name, user domain, serial number, and appendes them to a csv file (<i>WorkStation_Serial_Number.csv</i>). It was created to conduct a computer inventory audit during Covid to avoid direct exposure between auditor and end users.


 ```
Set-Location C:\Users\MySelf\Desktop
$Date= Get-Date
$SerialNumber= wmic bios get serialnumber
 
New-Object -TypeName PSCustomObject -Property @{
Computer_Name = $env:ComputerName
User_Name= $env:USERNAME
User_Domain= $env:USERDNSDOMAIN
Serial_Number =$SerialNumber[2]
Date = $Date
} | Export-Csv -Path WorkStation_Serial_Number.csv -NoTypeInformation -Append
 ```

The <i>Collect_Serial_Number.vbs</i> below allows each user to automatically run the <i>Serial_Number.ps1 .</i>

```
Set objShell = CreateObject("Wscript.shell")
objShell.run("powershell -executionpolicy bypass -file R:\Shared_Office\Serial_Number.ps1")
```

### Data Sanitization

The PowerShell script below , <i>cleaning_older_files.ps1</i>, deletes PII data stored in restricted folders with a LastWriteTime attribute older than 365 days. It also creates a log of the files deleted.

```
$Folder = "R:\SharedFolders\Reports"
#Delete all files older than 365 days
Get-ChildItem $Folder -Recurse -Force -ea 0  |
? {!$_.PsIsContainer -and $_.LastWriteTime -lt (Get-Date).AddDays(-365)} |
ForEach-Object {
   $_ | del -Force -Confirm:$false
   $_.FullName | Out-File "C:\Users\Desktop\deleted_log.txt" -Append }
```

### Daily Task Notification Email

The PowerShell script below, <i> email_notification.ps1,</i> checks and retrieves a Transaction Report(s). The report(s) is attached to an email which is already populated and send out daily if needed.On Mondays,it will also check for transaction reports arrived over the weekend.

```

Set-Location R:\Shared_Folder\Folder_to_Check
$subject_date= Get-Date -DisplayHint Date
$day_fullname = Get-Date -Format "dddd"
$Outlook = New-Object -ComObject Outlook.Application
$Mail = $Outlook.CreateItem(0)
$Mail.To = "employee01@staff.org"
$Mail.Subject = "Transaction Report "+$subject_date
$Mail.Body = "Good day,

Please find attached the Transaction Report. Let me know if you have any question.
"
$mail.save()
if ($day_fullname -eq "Monday"){
$filenames = Get-ChildItem *.txt| Where-Object {($_).LastWriteTime.Date  -ge ([datetime]::Today.AddDays(-2)
)}
}else {
$filenames = Get-ChildItem *.txt | Where-Object {($_).LastWriteTime.Date  -ge ([datetime]::Today
)}
}
for ($i=0; $i -lt $filenames.Count; $i++) {
 
        $outputfiles= $filenames[$i].FullName

        $mail.Attachments.Add($outputfiles)}

$inspector = $mail.GetInspector
$inspector.Display()
```

### Replicating Directory Structure Without Files
The PowerShell script below, copy_directory_structure, creates an identical directory structure (to include subdirectories) from one location to another rather than creating it manually. The script only replicates the directory structure and excludes any file.

```
$source="C\Users\MySelf"
$dest="R:\Shared_Folder\My_Section_Folder"
robocopy $source $dest /e /xf *.*
```

### Daily Reports Allocation
The PowerShell script below, <i> allocate_reports.ps1,</i> distributes daily reports to the appropriate folders. Rather than opening each file received, to determine its destination, end-user can rely on the script to properly identify each report based on their content, and place them in the final destination folder. The script runs on an infinite loop that starts all over every two hours.

```
do
{
    $reports_name='Worked_Hours','Evaluations','Awards','Pay_Status','Top_Perfomers','Allocations','Years_Of_Service'
    Select-String -List $reports_name 'C:\Main_System\Reports\*.txt' | move-item -LiteralPath {$_.Path }  -Destination 
    {"C:\Main_System\$($_.Pattern)"} -force 
    start-sleep -Seconds 7200 # 2 hours break before sarting the loop all over
    
}
until($infinity)
```

### Randomly select personnel records
The PowerShell script below,<i> random_records.ps1,</i> identifies 10 non duplicate personnel records for testing purposes. Records information is stored in a .csv file. Everytime records are identified, they are removed from the main roster and will not be participate in the next random audit. 


```
#setting the directory
$add_title=get-date -Format "MM/dd/yyyy/HH:mm"
Set-Location -path "R:\Restricted_Folder\Personnel_Roster.csv"
#importing the csv object
$document = Import-Csv .\Personnel_Roster.csv
#column to array
$column_name = @($document."Full Name")

#loop until 10 non duplicates sample are found.
do{
    $NonRepeat = $column_name|Get-Random -count 10
} until ($NonRepeat -notmatch '(\d)\k<1>')

#select full name and unit description from personnel roster
$final_sample = $document |select "Full_Name","Recent_Evaluation_Score,""Office_Desc"|? "Full_Name" -in $NonRepeat |export-csv random_records.csv -NoTypeInformation -Append
#deletes the name from the original 
$document = $document |select *| where "Full_ Name" -Notin $NonRepat |export-csv Personnel_Roster.csv -NoTypeInformation -force

```

### Find and Remove Identical Files
The PowerShell script below, <i> duplicate_files.ps1,</i> finds identical files based on the SHA1 algorithm, and moves them to <i> path_duplicates.</i>.SHA1 is in a hash or message digest algorithm where it generates 160-bit unique value from the input data. The input data size doesn’t matter as SHA1 always generates the same size message digest or hash which is 160 bit. 

```
$myFilePath='R:\Restricted_Folder\Personnel_Reports'
$path_duplicates= 'R:\Restricted_Folder\Personnel_Reports\Archive'
$fileHashes = Get-ChildItem -Path $myFilePath -Recurse -File | Get-Filehash -Algorithm SHA1
$duplicates = $fileHashes | Group hash | ? {$_.count -gt 1} | % {$_.Group} 

$uniqueItems = @{}
$doubledItems = @()

foreach($item in $duplicates) {
  
  if(-not $uniqueItems.ContainsKey($item.Hash)){
    $uniqueItems.Add($item.Hash,$item)
  }else{
    $doubledItems += $item
  }
}


# Remove indentical files
#$doubledItems | % {Remove-Item $_.path} -Verbose
# Move identical files
$doubledItems |% {move-item $_.path -destination $path_duplicates -Force}

```

### Remove Duplicate Rows Excel Document
The VBA subroutine below removes duplicate rows based on the content of multiple columns.


```
Sub remove_duplicates()

    Dim lastrow As Long

    With ThisWorkbook.Worksheets("Sheet1") 'the name of the Excel sheet
        If Application.WorksheetFunction.CountA(.Cells) <> 0 Then
            lastrow = .Cells.Find(What:="*", _
                          After:=.Range("A1"), _
                          Lookat:=xlPart, _
                          LookIn:=xlFormulas, _
                          SearchOrder:=xlByRows, _
                          SearchDirection:=xlPrevious, _
                          MatchCase:=False).Row
        Else
            lastrow = 1
        End If

        'Array(1, 2, 16) means 1 - for A, 2 for B and 16 for P columns
        .Range("A1:Q" & lastrow).RemoveDuplicates Columns:=Array(4, 5, 12), _
            Header:=xlYes 'if the data in column 4,5, and 12 is repeated in more than one row it will be removed.
    End With
End Sub

```

### Merging Multiple Excel Files
This VBA subroutine below allows user to merge multiple Excel Workbooks in one document.

```
Sub MergeExcelFiles()
    Dim fnameList, fnameCurFile As Variant
    Dim countFiles, countSheets As Integer
    Dim wksCurSheet As Worksheet
    Dim wbkCurBook, wbkSrcBook As Workbook
 
    fnameList = Application.GetOpenFilename(FileFilter:="Microsoft Excel Workbooks (*.xls;*.xlsx;*.xlsm),*.xls;*.xlsx;*.xlsm", Title:="Choose Excel files to merge", MultiSelect:=True)
 
    If (vbBoolean <> VarType(fnameList)) Then
 
        If (UBound(fnameList) > 0) Then
            countFiles = 0
            countSheets = 0
 
            Application.ScreenUpdating = False
            Application.Calculation = xlCalculationManual
 
            Set wbkCurBook = ActiveWorkbook
 
            For Each fnameCurFile In fnameList
                countFiles = countFiles + 1
 
                Set wbkSrcBook = Workbooks.Open(Filename:=fnameCurFile)
 
                For Each wksCurSheet In wbkSrcBook.Sheets
                    countSheets = countSheets + 1
                    wksCurSheet.Copy after:=wbkCurBook.Sheets(wbkCurBook.Sheets.Count)
                Next
 
                wbkSrcBook.Close SaveChanges:=False
 
            Next
 
            Application.ScreenUpdating = True
            Application.Calculation = xlCalculationAutomatic
 
            MsgBox "Processed " & countFiles & " files" & vbCrLf & "Merged " & countSheets & " worksheets", Title:="Merge Excel files"
        End If
 
    Else
        MsgBox "No files selected", Title:="Merge Excel files"
    End If
End Sub
```